In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

train_data.head()

train_data.describe()

test_data.head()

test_data.describe()

print('[train_data_unique]')
train_data.nunique()

print('[test_data_unique]')
test_data.nunique()

plt.figure(figsize=(14,7))
plt.hist(train_data['target'], bins = 50, fill = True, color = 'Skyblue')
plt.title('Target Distribution', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Target', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
print('mean target:', train_data['target'].mean())
print('std target:', train_data['target'].std())
print('median target:', train_data['target'].median())
plt.show()

plt.figure(figsize=(14,7))
plt.hist(train_data['standard_error'], bins = 50, fill = True, color = 'Skyblue')
plt.title('Standard Error Distribution', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Standard Error', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
print('mean standard error:', train_data['standard_error'].mean())
print('std standard error:', train_data['standard_error'].std())
print('median standard error:', train_data['standard_error'].median())
plt.show()

plt.figure(figsize=(14,7))
plt.scatter(train_data['target'], train_data['standard_error'], color = 'Skyblue')
plt.title('target vs standard error', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('target', fontsize=16)
plt.ylabel('standard error', fontsize=16)
print('id:', train_data.iloc[106, 0], 'is (0,0)')
print('[excerpt]')
print(train_data.iloc[106, 3])
plt.show()

train_data_url = train_data.iloc[:, 0:2]
train_data_url.iloc[:, 1] = train_data_url.iloc[:, 1].fillna('blank')
train_data_url = train_data_url.replace('https://', '', regex=True)
train_data_url = train_data_url.replace('http://', '', regex=True)
for i in range(len(train_data_url['id'])):
    if train_data_url.iloc[i, 1] != 'blank':
        end = train_data_url.iloc[i, 1].index('/')
        train_data_url.iloc[i, 1] = train_data_url.iloc[i, 1][:end]

train_data_url = train_data_url.rename(columns={'url_legal':'url_legal_domain'})
train_data_url.insert(1, 'url_legal', train_data['url_legal'])
train_data_url['license'] = train_data['license']
train_data_url['excerpt'] = train_data['excerpt']
train_data_url['target'] = train_data['target']
train_data_url['standard_error'] = train_data['standard_error']
train_data_url.iloc[:, 1] = train_data_url.iloc[:, 1].fillna('blank')
train_data_url.iloc[:, 3] = train_data_url.iloc[:, 3].fillna('blank')
print(train_data_url)

plt.figure(figsize=(14,7))
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.title('url_legal_domain vs target', fontsize=16)
plt.xlabel('url_legal_domain', fontsize=16)
plt.ylabel('target', fontsize=16)
sns.boxplot(x='url_legal_domain', y='target', data=train_data_url)

train_data_url_sorted = pd.DataFrame(train_data_url.groupby('url_legal_domain').mean()['target'])
print(train_data_url_sorted.sort_values('target'))
plt.show()

plt.figure(figsize=(14,7))
plt.title('url_legal_domain', fontsize=16)
plt.xticks(fontsize=14, rotation=90)
plt.yticks(fontsize=14)
plt.xlabel('url_legal_domain', fontsize=16)
plt.ylabel('target', fontsize=16)
sns.barplot(x='url_legal_domain', y='target', data=train_data_url)

train_data_url_sorted = pd.DataFrame(train_data_url.groupby('url_legal_domain').mean()['target'])
print(train_data_url_sorted.sort_values('target'))
plt.show()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
